# Importe

In [2]:
# general
import pandas as pd

# waits
import random
from time import sleep

# driver
import undetected_chromedriver.v2 as uc
from selenium import webdriver
import chromedriver_autoinstaller

# selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# other
# from ssl import Options

# multi
from concurrent import futures


# Driver

In [3]:
def get_driver(ip="", port="", flag_proxy=False):
    # PATH = "C:\Program Files (x86)\chromedriver.exe"
    chrome_options = uc.ChromeOptions()

    # Removes navigator.webdriver flag
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    # prevent bugs and skip popups
    chrome_options.add_argument('--no-first-run --no-service-autorun --password-store=basic')
    chrome_options.add_argument("--disable-extensions")

    # uc - user agent
    # bot detection can determine if os in user-agent differs from real os
    # chrome_options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15")
    # chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

    # add multiple download flag
    dl_multi = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", dl_multi)

    # add download location
    dl_path = {"download.default_directory" : "C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\02_Download"}
    chrome_options.add_experimental_option("prefs", dl_path)

    # add profile
    # uc.Chrome(user_data_dir='C:\\Users\\lange\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
    
    # add proxy if True
    # -1 to use first element after use of driver to scrape proxies
    if flag_proxy == True:       
        PROXY = str(ip + ":" + port)
        print(PROXY)
        chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

    # # be careful with headless browsing
    # chrome_options.add_argument("--headless")
    
    # # additional options to allow downloading while in headless mode
    # prefs = {
    # "download.default_directory": "C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\02_Download",
    # "download.directory_upgrade": True,
    # "download.prompt_for_download": False,
    # "plugins.always_open_pdf_externally": True,
    # "profile.default_content_settings.popups": "0",
    # "profile.content_settings.exceptions.automatic_downloads.*.setting": "1"
    # }
    # args = ["--no-sandbox",
    #         "--disable-dev-shm-usage"]
    
    # chrome_options.add_experimental_option("prefs", prefs)
    # chrome_options.add_experimental_option("args", args)
    # # chrome_options.add_experimental_option('useAutomationExtension', False)

    # webdriver vs. undetected chromedriver
    # driver = webdriver.Chrome(executable_path=PATH, options=chrome_options)
    driver = uc.Chrome(options=chrome_options)
    driver.maximize_window()
    
    return driver

# Scraper

In [1]:
def scrape(driver, URL, regart, regnum, regricht):
   
    driver.implicitly_wait(10)
    driver.get(URL)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    try:       
        # Input Felder bestimmen
        input_regart = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "form:registerArt_label")))
        input_regnum = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "form:registerNummer")))
        input_regricht = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "form:registergericht_label")))
        
        # Dropdown regart
        input_regart.click()
        drops_regart = driver.find_elements(By.XPATH, "/html/body/div[4]/div/ul/li")
        
        sleep(round(random.uniform(1.31, 3.11), 2))
        for element in drops_regart:
            innerhtml = element.get_attribute('innerHTML')
            if innerhtml == regart:
                element.click()
                break
        
        # Input Registernummer
        sleep(round(random.uniform(1.31, 3.11), 2))
        input_regnum.clear()
        input_regnum.send_keys(regnum)
        
        # Dropdown regricht
        input_regricht.click()
        drops_regricht = driver.find_elements(By.XPATH, "/html/body/div[5]/div[2]/ul/li")
        
        sleep(round(random.uniform(1.31, 3.11), 2))
        for element in drops_regricht:
            innerhtml = element.get_attribute('innerHTML')
            if innerhtml == regricht:
                element.click()
                break

        # submit funktioniert nicht, daher über Suche Button
        sleep(round(random.uniform(1.31, 3.11), 2))
        driver.find_element(By.ID, "form:btnSuche").click()

        # durch Suche nach
        sleep(round(random.uniform(1.31, 3.11), 2))
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.LINK_TEXT, "AD"))).click()
        sleep(round(random.uniform(1.31, 3.11), 2))
        driver.find_element(By.ID, "form:kostenpflichtigabrufen").click()       
        status = 1
        
    except Exception as exc:
        print("ERROR Scraper\nregart: {0}\nregnum: {1}\nregricht: {2}\n{3}".format(regart, regnum, regricht, exc))
        status = 0
        sleep(round(random.uniform(60.31, 70.11), 2))
        return driver, status
    
    sleep(round(random.uniform(60.31, 70.11), 2))
    return driver, status

# Data

In [4]:
# Einlesen der gelabelten URLs, samt Registerinformationen zur Suche im Handelsregister
df_firms = pd.read_excel("C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\UN_Branche_Sample.xlsx")

In [5]:
df_firms.dtypes

Firmierung lt Webseite    object
Amtsgericht               object
Registerart               object
Nummer                    object
URL                       object
Branche                   object
dtype: object

In [6]:
df_firms.head()

,Firmierung lt Webseite,Amtsgericht,Registerart,Nummer,URL,Branche
0,AUDI AG,Ingolstadt,HRB,1,https://www.audi.de/,Automotive
1,Volkswagen AG,Braunschweig,HRB,100484,https://www.volkswagen.de/,Automotive
2,ŠKODA AUTO Deutschland GmbH,Darmstadt,HRB,5242,https://www.skoda-auto.de/,Automotive
3,Tesla Germany GmbH,Berlin (Charlottenburg),HRB,235261,https://www.tesla.com/de_de/,Automotive
4,BMW AG,München,HRB,42243,https://www.bmw.de/,Automotive


In [7]:
# prepare df_firms
df_firms["download"] = ""

df_firms = df_firms.apply(lambda x: x.apply(str) if x.dtype == 'object' else x)
df_firms = df_firms.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Calls

In [8]:
# call

driver = get_driver()
URL = "https://www.handelsregister.de/rp_web/erweitertesuche.xhtml"
# URL = "https://bot.sannysoft.com/"    # test site for bot detection
# URL = "https://nowsecure.nl"          # test site with max anti-bot protection

for index, row in df_firms.iterrows():
    status = 2
    driver, status = scrape(driver, URL, regart=row["regart"], regnum=row["regnum"], regricht=row["regricht"])
    row["download"] = status
    
driver.quit()

ERROR Scraper
regart: HRB
regnum: 10
regricht: Stuttgart
Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00F5ACD3+2075859]
	Ordinal0 [0x00EEEE61+1633889]
	Ordinal0 [0x00DEB7BD+571325]
	Ordinal0 [0x00E1AC2F+764975]
	Ordinal0 [0x00E1AE1B+765467]
	Ordinal0 [0x00E4D0F2+970994]
	Ordinal0 [0x00E37364+881508]
	Ordinal0 [0x00E4B56A+963946]
	Ordinal0 [0x00E37136+880950]
	Ordinal0 [0x00E0FEFD+720637]
	Ordinal0 [0x00E10F3F+724799]
	GetHandleVerifier [0x0120EED2+2769538]
	GetHandleVerifier [0x01200D95+2711877]
	GetHandleVerifier [0x00FEA03A+521194]
	GetHandleVerifier [0x00FE8DA0+516432]
	Ordinal0 [0x00EF682C+1665068]
	Ordinal0 [0x00EFB128+1683752]
	Ordinal0 [0x00EFB215+1683989]
	Ordinal0 [0x00F06484+1729668]
	BaseThreadInitThunk [0x76B46939+25]
	RtlGetFullPathName_UEx [0x77CF8FD2+1218]
	RtlGetFullPathName_UEx [0x77CF8F9D+1165]

ERROR Scraper
regart: HRB
regnum: 28 163
regricht: Frankfurt am Main
Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00F5ACD3+2075859]
	Ordinal0 [0x00EEEE61+1633889]
	Ordinal0 